# 自定义调试体验文档

[![](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/master/docs/notebook/mindspore_custom_debugging_info.ipynb)

## 概述

本文将使用[快速入门](https://gitee.com/mindspore/docs/blob/master/docs/sample_code/lenet/lenet.py)作为样例，并通过构建自定义调试函数：`Callback`、`metrics`、Print算子、日志打印、数据Dump功能等，同时将构建的自定义调试函数添加进代码中，通过运行效果来展示具体如何使用MindSpore提供给我们的自定义调试能力，帮助快速调试训练网络。
体验过程如下：

1. 数据准备。
2. 定义深度神经网络LeNet5。
3. 使用Callback回调函数构建StopAtTime类来控制训练停止时间。
4. 设置日志环境变量。
5. 启动同步Dump功能。
5. 定义训练网络并执行训练。
6. 执行测试。
7. 算子输出数据的读取与展示。

> 本次体验适用于GPU环境。

## 数据准备

### 数据集的下载

这里我们需要将MNIST数据集中随机取出一张图片，并增强成适合LeNet网络的数据格式（如何处理请参考[初学入门](https://www.mindspore.cn/tutorials/zh-CN/master/quick_start.html)）

以下示例代码将数据集下载并解压到指定位置。

In [ ]:
import os
import requests

def download_dataset(dataset_url, path):
    filename = dataset_url.split("/")[-1]
    save_path = os.path.join(path, filename)
    if os.path.exists(save_path):
        return
    if not os.path.exists(path):
        os.makedirs(path)
    res = requests.get(dataset_url, stream=True, verify=False)
    with open(save_path, "wb") as f:
        for chunk in res.iter_content(chunk_size=512):
            if chunk:
                f.write(chunk)

train_path = "datasets/MNIST_Data/train"
test_path = "datasets/MNIST_Data/test"

download_dataset("https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/train-labels-idx1-ubyte", train_path)
download_dataset("https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/train-images-idx3-ubyte", train_path)
download_dataset("https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/t10k-labels-idx1-ubyte", test_path)
download_dataset("https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/t10k-images-idx3-ubyte", test_path)

下载的数据集文件的目录结构如下：

```text
./datasets/MNIST_Data
├── test
│   ├── t10k-images-idx3-ubyte
│   └── t10k-labels-idx1-ubyte
└── train
    ├── train-images-idx3-ubyte
    └── train-labels-idx1-ubyte
```

`custom_debugging_info.ipynb`为本文文档。

### 数据集的增强操作

下载的数据集，需要通过`mindspore.dataset`处理成适用于MindSpore框架的数据，再使用一系列框架中提供的工具进行数据增强操作来适应LeNet网络的数据处理需求。

In [2]:
import mindspore.dataset as ds
import mindspore.dataset.vision.c_transforms as CV
import mindspore.dataset.transforms.c_transforms as C
from mindspore.dataset.vision import Inter
from mindspore import dtype as mstype

def create_dataset(data_path, batch_size=32, repeat_size=1,
                   num_parallel_workers=1):
    """ create dataset for train or test
    Args:
        data_path (str): Data path
        batch_size (int): The number of data records in each group
        repeat_size (int): The number of replicated data records
        num_parallel_workers (int): The number of parallel workers
    """
    # define dataset
    mnist_ds = ds.MnistDataset(data_path)

    # define operation parameters
    resize_height, resize_width = 32, 32
    rescale = 1.0 / 255.0
    shift = 0.0
    rescale_nml = 1 / 0.3081
    shift_nml = -1 * 0.1307 / 0.3081

    # define map operations
    trans_image_op = [
        CV.Resize((resize_height, resize_width), interpolation=Inter.LINEAR),
        CV.Rescale(rescale_nml, shift_nml),
        CV.Rescale(rescale, shift),
        CV.HWC2CHW()
    ]
    type_cast_op = C.TypeCast(mstype.int32)

    # apply map operations on images
    mnist_ds = mnist_ds.map(operations=type_cast_op, input_columns="label", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=trans_image_op, input_columns="image", num_parallel_workers=num_parallel_workers)

    # apply DatasetOps
    buffer_size = 10000
    mnist_ds = mnist_ds.shuffle(buffer_size=buffer_size)
    mnist_ds = mnist_ds.batch(batch_size, drop_remainder=True)
    mnist_ds = mnist_ds.repeat(repeat_size)

    return mnist_ds

## 定义深度神经网络LeNet5

针对MNIST数据集我们采用的是LeNet5网络，先对卷积函数和全连接函数初始化，然后`construct`构建神经网络。

In [3]:
from mindspore.common.initializer import Normal
import mindspore.nn as nn

class LeNet5(nn.Cell):
    """Lenet network structure."""
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, pad_mode="valid")
        self.conv2 = nn.Conv2d(6, 16, 5, pad_mode="valid")
        self.fc1 = nn.Dense(16 * 5 * 5, 120, weight_init=Normal(0.02))
        self.fc2 = nn.Dense(120, 84, weight_init=Normal(0.02))
        self.fc3 = nn.Dense(84, 10)
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

    def construct(self, x):
        x = self.max_pool2d(self.relu(self.conv1(x)))
        x = self.max_pool2d(self.relu(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## 构建自定义回调函数StopAtTime

使用回调函数的基类Callback，构建训练定时器`StopAtTime`，其基类（可在源码中找到位置在`/mindspore/nn/callback`）为：

```python
class Callback():
    def begin(self, run_context):
        pass
    def epoch_begin(self, run_context):
        pass
    def epoch_end(self, run_context):
        pass
    def step_begin(self, run_context):
        pass
    def step_end(self, run_context):
        pass
    def end(self, run_context):
        pass
```

- `begin`：表示训练开始时执行。
- `epoch_begin`：表示每个epoch开始时执行。
- `epoch_end`：表示每个epoch结束时执行。
- `step_begin`：表示每个step刚开始时执行。
- `step_end`：表示每个step结束时执行。
- `end`：表示训练结束时执行。

了解上述基类的用法后，还有一个参数`run_context`，这是一个类，存储了模型训练中的各种参数，我们在这里使用`print(cb_params.list_callback)`将其放在`end`中打印（当然也可以使用`print(cb_param)`打印所有参数信息，由于参数信息太多，我们这里只选了一个参数举例），后续在执行完训练后，根据打印信息，会简单介绍`run_context`类中各参数的意义，我们开始构建训练定时器，如下：

In [4]:
from mindspore.train.callback import Callback
import time

class StopAtTime(Callback):
    def __init__(self, run_time):
        super(StopAtTime, self).__init__()
        self.run_time = run_time*60

    def begin(self, run_context):
        cb_params = run_context.original_args()
        cb_params.init_time = time.time()

    def step_end(self, run_context):
        cb_params = run_context.original_args()
        epoch_num = cb_params.cur_epoch_num
        step_num = cb_params.cur_step_num
        loss = cb_params.net_outputs
        cur_time = time.time()
        if (cur_time - cb_params.init_time) > self.run_time:
            print("epoch: ", epoch_num, " step: ", step_num, " loss: ", loss)
            run_context.request_stop()
    def end(self, run_context):
        cb_params = run_context.original_args()
        print(cb_params.list_callback)

## 启动同步Dump功能

本例中使用同步Dump功能，导出每次迭代中前向传播和反向传播算子的输出数据，导出的数据方便用户在进行优化训练策略时进行分析使用，如需导出更多数据可参考[官方教程](https://www.mindspore.cn/docs/programming_guide/zh-CN/master/custom_debugging_info.html#dump)。

In [5]:
import os
import json

abspath = os.getcwd()

data_dump = {
        "common_dump_settings": {
            "dump_mode": 0,
            "path": abspath + "/data_dump",
            "net_name": "LeNet5",
            "iteration": "0|5-8|100-120",
            "input_output": 2,
            "kernels": ["Default/network-WithLossCell/_backbone-LeNet5/flatten-Flatten/Reshape-op118"],
            "support_device": [0, 1, 2, 3, 4, 5, 6, 7]
        },
        "e2e_dump_settings": {
            "enable": True,
            "trans_flag": False
        }
}

with open("./data_dump.json", "w", encoding="GBK") as f:
    json.dump(data_dump, f)

os.environ['MINDSPORE_DUMP_CONFIG'] = abspath + "/data_dump.json"

执行完上述命令后会在工作目录上生成`data_dump.json`文件，目录结构如下：

```text
.
└── data_dump.json

```

启动同步Dump功能需要注意：

- `path`需要设置成绝对路径。例如`/usr/data_dump`可以，`./data_dump`则不行。
- `e2e_dump_settings`中的`enable`需要设置成`True`。

- 需要将生成的`data_dump.json`文件添加至系统环境变量中。

## 设置日志环境变量

MindSpore采用`glog`来输出日志，我们这里将日志输出到屏幕：

`GlOG_v`：控制日志的级别，默认值为2，即WARNING级别，对应关系如下：0-DEBUG、1-INFO、2-WARNING、3-ERROR、4-CRITICAL。本次设置为1。

`GLOG_logtostderr`：控制日志输出方式，设置为`1`时，日志输出到屏幕；值设置为`0`时，日志输出到文件。设置输出屏幕时，日志部分的信息会显示成红色，设置成输出到文件时，会在`GLOG_log_dir`路径下生成`mindspore.log`文件。

> 更多设置请参考官网：<https://www.mindspore.cn/docs/programming_guide/zh-CN/master/custom_debugging_info.html>

In [7]:
import os
from mindspore import log as logger

os.environ['GLOG_v'] = '1'
os.environ['GLOG_logtostderr'] = '1'
os.environ['GLOG_log_dir'] = 'D:/' if os.name == "nt" else '/var/log/mindspore'
os.environ['logger_maxBytes'] = '5242880'
os.environ['logger_backupCount'] = '10'
print(logger.get_log_config())

{'GLOG_v': '1', 'GLOG_logtostderr': '1'}


打印信息为`GLOG_v`的等级：`INFO`级别。

输出方式`GLOG_logtostderr`：`1`表示屏幕输出。

## 定义训练网络并执行训练

### 定义训练网络

此过程中先将之前生成的模型文件`.ckpt`和`.meta`的数据删除，并将模型需要用到的参数配置到`Model`。

In [8]:
from mindspore import context, Model
from mindspore.nn import SoftmaxCrossEntropyWithLogits
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, LossMonitor

# clean files
if os.name == "nt":
    os.system('del/f/s/q *.ckpt *.meta')
else:
    os.system('rm -f *.ckpt *.meta *.pb')

context.set_context(mode=context.GRAPH_MODE, device_target="GPU")
lr = 0.01
momentum = 0.9
epoch_size = 3
train_data_path = "./datasets/MNIST_Data/train"
eval_data_path = "./datasets/MNIST_Data/test"
model_path = "./models/ckpt/custom_debugging_info/"

net_loss = SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")
repeat_size = 1
network = LeNet5()

metrics = {
    'accuracy': nn.Accuracy(),
    'loss': nn.Loss(),
    'precision': nn.Precision(),
    'recall': nn.Recall(),
    'f1_score': nn.F1()
    }
net_opt = nn.Momentum(network.trainable_params(), lr, momentum)

config_ck = CheckpointConfig(save_checkpoint_steps=1875, keep_checkpoint_max=10)

ckpoint_cb = ModelCheckpoint(prefix="checkpoint_lenet", directory=model_path, config=config_ck)

model = Model(network, net_loss, net_opt, metrics=metrics)

### 执行训练

在构建训练网络中，给`model.train`传入了三个回调函数，分别是`ckpoint_cb`，`LossMonitor`，`stop_cb`；其分别代表如下：

`ckpoint_cb`：即是`ModelCheckpoint`，设置模型保存的回调函数。

`LossMonitor`：loss值监视器，打印训练过程每步的loss值。

`stop_cb`：即是`StopAtTime`，上面刚构建的训练定时器。

我们将训练定时器`StopAtTime`设置成36秒，即`run_time=0.6`。

In [9]:
print("============== Starting Training ==============")
ds_train = create_dataset(train_data_path, repeat_size=repeat_size)
stop_cb = StopAtTime(run_time=0.6)
model.train(epoch_size, ds_train, callbacks=[ckpoint_cb, LossMonitor(375), stop_cb], dataset_sink_mode=False)

============== Starting Training ==============


[INFO] ME(14134:140231287715648,MainProcess):2020-12-01-17:12:19.263.012 [mindspore/train/serialization.py:379] Execute save the graph process.
[INFO] ME(14134:140231287715648,MainProcess):2020-12-01-17:12:29.689.876 [mindspore/train/serialization.py:168] Execute save checkpoint process.
[INFO] ME(14134:140231287715648,MainProcess):2020-12-01-17:12:29.704.062 [mindspore/train/serialization.py:214] Save checkpoint process finish.


epoch:  1  step:  30  loss:  2.3048654
[<mindspore.train.callback._checkpoint.ModelCheckpoint object at 0x7f8a1c116350>, <mindspore.train.callback._loss_monitor.LossMonitor object at 0x7f8997005150>, <__main__.StopAtTime object at 0x7f8a1f042950>]


以上打印信息中，主要分为两部分：

- 日志信息部分：

    - `[INFO]`部分信息即为日志输出的信息，由于没有Warning信息，目前主要记录的是训练的几个重要步骤。

- 回调函数信息部分：

    - `LossMonitor`：每步的loss值。
    - `StopAtTime`：在每个epoch结束及训练时间结束时，打印当前epoch的训练总时间(单位为毫秒)，每步训练花费的时间以及平均loss值，另外在训练结束时还打印了`run_context.list_callback`的信息，这条信息表示本次训练过程中使用的回调函数；另外`run_conext.original_args`中还包含以下参数：

        - `train_network`：网络的各类参数。
        - `epoch_num`：训练的epoch数。
        - `batch_num`：一个epoch的step数。
        - `mode`：MODEL的模式。
        - `loss_fn`：使用的损失函数。
        - `optimizer`：使用的优化器。
        - `parallel_mode`：并行模式。
        - `device_number`：训练卡的数量。
        - `train_dataset`：训练的数据集。
        - `list_callback`：使用的回调函数。
        - `train_dataset_element`：打印当前batch的数据集。
        - `cur_step_num`：当前训练的step数。
        - `cur_epoch_num`：当前的epoch。
        - `net_outputs`：网络返回值。

   几乎在训练中的所有重要数据，都可以从Callback中取得，所以Callback也是在自定义调试中比较常用的功能。

## 执行测试

测试网络中我们的自定义函数`metrics`将在`model.eval`中被调用，除了模型的预测正确率外`recall`，`F1`等不同的检验标准下的预测正确率也会打印出来：

In [10]:
print("============== Starting Testing ==============")
ds_eval = create_dataset(eval_data_path, repeat_size=repeat_size)
acc = model.eval(ds_eval, dataset_sink_mode=False)
print("============== Accuracy:{} ==============".format(acc))

============== Starting Testing ==============
============== Accuracy:{'accuracy': 0.10106169871794872, 'loss': 2.302597688558774, 'precision': array([0.       , 0.       , 0.       , 0.1010617, 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       ]), 'recall': array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]), 'f1_score': array([0.        , 0.        , 0.        , 0.18357136, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])} ==============


`Accuracy`部分的信息即为`metric`控制输出的信息，模型的预测值正确率和其他标准下验证（0-9）的正确率值，至于不同的验证标准计算方法，大家可以去官网搜索`mindspore.nn`查找，这里就不多介绍了。

## 算子输出数据的读取展示

执行完成上述训练后，可以在`data_dump`文件夹中找到导出的训练数据，按照本例`data_dump.json`文件的设置，在目录`data_dump/LeNet5/device_0/`中找到每次迭代的数据，保存每次迭代的数据文件夹名称为`iteration_{迭代次数}`，每个算子输出数据的文件后缀为`.bin`，可以使用`numpy.fromfile`读取其中的数据。

本例子，在第400次迭代数据中，随机读取其中一个算子的输出文件并进行展示：

In [11]:
import numpy as np
import random

dump_data_path = "./data_dump/LeNet5/device_0/iteration_400/"
ops_output_file = random.choice(os.listdir(dump_data_path))
print("ops name:", ops_output_file, "\n")
ops_dir = dump_data_path + ops_output_file
ops_output = np.fromfile(ops_dir)
print("ops output value:", ops_output, "\n")
print("the shape of ops output:", ops_output.shape)

ops name: fc2.weight_output_0_shape_84_120_Float32_DefaultFormat.bin 

ops output value: [-1.86227040e-17  7.49122057e-21 -5.01539318e-16 ... -6.28152809e-20
  7.43756225e-16  3.97661325e-20] 

the shape of ops output: (5040,)


## 总结

本例使用了MNIST数据集，通过LeNet5神经网络进行训练，将自定义调试函数结合到代码中进行调试，同时展示了使用方法和部分功能，并使用调试函数导出需要的输出数据，来更好的认识自定义调试函数的方便性，以上就是本次的体验内容。